In [3]:
import pandas as pd

In [4]:
# задача: посчитать окупаемость маркетинга за последние 12 месяцев

# тратим деньги
# привлекаем пользователей
# пользователи что-то покупают

# когорты от даты регистрации
# посчитаем 
# - выручку и LTV
# - рекламные расходы
# - окупаемость маркетинга

In [5]:
# Алгоритм
# 0. Формулируем задачу
# 1. Определяем когорту (событие и временной промежуток) — даты регистрации по месяцам
# 2. Выделяем целевые метрики - выручка, LTV, ROAS
# 3. Выбираем подходящий формат отчета - возрастной

In [20]:
ads = pd.read_csv('data/ads.csv')
ads['dt'] = pd.to_datetime(ads['dt'])
ads['month'] = ads['dt'].dt.to_period('M')

monthly_ads = ads.groupby('month').agg(ad_spend = ('ad_spend', 'sum'))

monthly_ads.head()

,ad_spend
month,
2019-11,43.400000
2019-12,3052.600000
2020-01,10912.514286
2020-02,13781.914286
2020-03,21480.142857


In [7]:
users = pd.read_csv('data/users.csv', index_col='id')
users['reg_date'] = pd.to_datetime(users['reg_date'])
users['reg_month'] = users['reg_date'].dt.to_period('M')
users[['reg_date', 'reg_month']]

,reg_date,reg_month
id,,
1124,2020-06-28,2020-06
1766,2020-09-20,2020-09
4132,2021-09-01,2021-09
3662,2021-06-23,2021-06
4804,2021-12-15,2021-12
...,...,...
3895,2021-07-27,2021-07
4235,2021-09-21,2021-09
1776,2020-09-22,2020-09


In [8]:
cohort_size = users.groupby('reg_month').agg(n_users = ('email', 'count')).reset_index()

In [9]:
orders = pd.read_csv('data/orders.csv', index_col='payment_id')
orders['created_at'] = pd.to_datetime(orders['created_at'])
orders = (
    orders
    .groupby('user_id')
    .agg(first_payment_at = ('created_at','min'))
    .merge(orders, how='inner', left_index=True, right_on='user_id')
)
orders['payment_month'] = orders['created_at'].dt.to_period('M')
orders['first_payment_month'] = orders['first_payment_at'].dt.to_period('M')

orders = orders.merge(
    users[['reg_date', 'reg_month']], 
    how='inner', 
    left_on='user_id', 
    right_index=True)

orders.head()

orders['cohort_age_days'] = (orders['created_at'] - orders['reg_date']).dt.days
orders['cohort_age_months'] = orders['cohort_age_days'] // 30

In [10]:
orders.reg_month.max()

Period('2022-11', 'M')

In [11]:
cohort_size.tail()

,reg_month,n_users
31,2022-07,276
32,2022-08,235
33,2022-09,233
34,2022-10,213
35,2022-11,165


In [27]:
cohorts = (
    orders
    .groupby(['reg_month', 'cohort_age_months'])
    .agg(revenue = ('payment_sum', 'sum'))
    .reset_index()
)
cohorts = cohorts.merge(cohort_size, how='outer', on='reg_month')
cohorts['rev_per_user'] = cohorts['revenue'] / cohorts['n_users']

cohorts['ltv'] = cohorts.groupby('reg_month')['rev_per_user'].expanding().sum().values


cohorts = (
    cohorts
    .merge(
        monthly_ads, 
        how='left', 
        left_on = 'reg_month',
        right_index=True)
)


cohorts['cac'] = cohorts['ad_spend'] / cohorts['n_users']
cohorts['roas'] = cohorts['ltv'] / cohorts['cac']

In [28]:
cohorts.head()

,reg_month,cohort_age_months,revenue,n_users,rev_per_user,ltv,ad_spend,cac,roas
0,2019-12,0,533.38,22,24.244545,24.244545,3052.6,138.754545,0.174730
1,2019-12,1,79.06,22,3.593636,27.838182,3052.6,138.754545,0.200629
2,2019-12,2,46.81,22,2.127727,29.965909,3052.6,138.754545,0.215963
3,2019-12,3,151.29,22,6.876818,36.842727,3052.6,138.754545,0.265524
4,2019-12,4,251.66,22,11.439091,48.281818,3052.6,138.754545,0.347966


In [30]:
pd.pivot(
    cohorts,
    index='reg_month',
    columns='cohort_age_months',
    values='roas'
).fillna('')

cohort_age_months,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
reg_month,,,,,,,,,,,,,,,,,,,,,
2019-12,0.174730,0.200629,0.215963,0.265524,0.347966,0.362792,0.376885,0.384685,0.43384,0.468473,...,0.682628,0.698359,,0.714132,,,,0.730171,,
2020-01,0.307316,0.359937,0.399143,0.456941,0.516587,0.589149,0.644029,0.692543,0.717337,0.757847,...,1.014302,,1.030458,1.035515,1.039461,1.040528,1.047414,1.056428,1.057841,1.059321
2020-02,0.249810,0.30665,0.366482,0.401173,0.452644,0.488162,0.521664,0.570646,0.602748,0.623598,...,0.830014,0.835481,0.837686,0.841121,0.843939,0.844675,0.846301,0.860909,,
2020-03,0.152534,0.18602,0.212057,0.250832,0.292465,0.330256,0.366041,0.398168,0.423891,0.437489,...,0.559463,0.571688,0.57815,0.581836,0.585144,0.588755,0.59061,,,
2020-04,0.124964,0.16535,0.200234,0.231479,0.249526,0.276562,0.293393,0.311888,0.325146,0.343544,...,0.436016,0.439618,0.445857,0.44748,0.455209,,,,,
2020-05,0.146635,0.177877,0.216219,0.243101,0.267205,0.284266,0.311944,0.332037,0.341365,0.348471,...,0.440521,0.442452,0.445159,0.447344,0.449144,,,,,
2020-06,0.131692,0.169852,0.202432,0.220804,0.241515,0.256709,0.271248,0.28232,0.296799,0.302263,...,0.38966,0.390474,0.39166,,,,,,,
2020-07,0.139069,0.168392,0.191131,0.216355,0.244151,0.265875,0.28943,0.297467,0.316207,0.334978,...,0.433558,0.435641,0.436591,,,,,,,
2020-08,0.107892,0.125659,0.155915,0.172556,0.189899,0.202839,0.215945,0.22821,0.237575,0.252341,...,0.326289,,,,,,,,,
